In [ ]:
import sys 
sys.path.append('~/Desktop/ENV_AGENT1')
from simple_gridworld import *
import numpy as np
import torch
import matplotlib.pyplot as plt 
import torch.optim as optim
import copy
import os

## Random Policy.

In [ ]:
def policy():
    return np.random.randint(0,4) 

env=SimpleGridWorld()
episodes=1
for _ in range(episodes):
    obs=env.reset()
    cumulative_reward=0
    while True:
        obs,reward,trunc=env.action_step(policy())
        cumulative_reward+=reward
        if trunc:
            break 
    print(f'Cumulative Reward {cumulative_reward}')

## DQN Policy.

In [ ]:
import model as M

In [ ]:
PROJECT_PATH="/home/trithankar-mittra/Desktop/Env_Agent1"
!rm -rf "{PROJECT_PATH}/save_state"
!mkdir "{PROJECT_PATH}/save_state"
!mkdir "{PROJECT_PATH}/results"

In [ ]:
mx_timeSteps=25 # agent's expectation of how long each episode will last.
env = SimpleGridWorld()
num_actions=4
update_after_actions=2**6
update_target_network=2**8
frame_cnt=0
episode_reward_hist,episode_avg_reward=[],[]
epsilon,epsilon_min,epsilon_max=1.0,0.005,1.0 
epsilon_interval = (
    epsilon_max - epsilon_min
) 

# Expected number of frames 
episodes=10000
total_frames=mx_timeSteps*episodes
# New logic.
epsilon_greedy=int(0.2*total_frames)
epsilon_greedy_frames=int((50/95)*total_frames) # After 50% of total frames I want the value of epsilon to reduce to 5/100 from 1.

batch_size=2**10
action=-1
replay_mem={'state':[],'next_state':[],'action':[],'reward':[]}
mx_replay_memory=100000
gamma=0.9
# MODEL
save_step=2**10;load_from_checkpoint=False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs=10
first=True

In [ ]:
episodes=10000
load_from_checkpoint=True

In [ ]:
model=M.NNModel().to(device)
model_target=M.NNModel().to(device)
for param in model.parameters():
    param.data.fill_(0)
for param in model_target.parameters():
    param.data.fill_(0)
if load_from_checkpoint==True:
    model.load_state_dict(torch.load('results/model.ml'))
    model_target.load_state_dict(torch.load('results/model.ml'))
    episode_reward_hist=np.load(f'results/ep_tot.npy').tolist()
    episode_avg_reward=np.load(f'results/ep_avg.npy').tolist()
    replay_mem['state']=np.load(f'results/state.npy').tolist()
    replay_mem['next_state']=np.load(f'results/next_state.npy').tolist()
    replay_mem['action']=np.load(f'results/action.npy').tolist()
    replay_mem['reward']=np.load(f'results/reward.npy').tolist()
    file_read=open('results/frame_cnt.dat','r')
    frame_cnt=int(file_read.read())
    file_read.close()
    file_read=open('results/epsilon.dat','r')
    epsilon=float(file_read.read())
    file_read.close()
if first:   
    load_from_checkpoint=True
    first=False
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
for _ in range(episodes):
    episode_reward=0
    state = env.reset()
    for __ in range(1,mx_timeSteps):
        frame_cnt+=1
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)
        replay_mem['state'].append(state)
        if frame_cnt<=epsilon_greedy or epsilon>np.random.rand(1)[0]:
            action=np.random.choice(num_actions)
        else:
            action=torch.argmax(M.predict(model,[state],device))
            action=action.item()
        next_state,reward,terminated=env.action_step(action)
        episode_reward+=reward 
        state=copy.deepcopy(next_state)
        replay_mem['next_state'].append(next_state)
        replay_mem['action'].append(action)
        replay_mem['reward'].append(reward)
        if (frame_cnt%save_step==0):
            print(f'Saving ...')
            np.save(f'results/ep_tot.npy',np.array(episode_reward_hist))
            np.save(f'results/ep_avg.npy',np.array(episode_avg_reward))
            np.save(f'results/state.npy',np.array(replay_mem['state']))
            np.save(f'results/next_state.npy',np.array(replay_mem['next_state']))
            np.save(f'results/action.npy',np.array(replay_mem['action']))
            np.save(f'results/reward.npy',np.array(replay_mem['reward']))
            file_write=open('results/frame_cnt.dat','w')
            file_write.write(str(frame_cnt))
            file_write.close()
            file_write=open('results/epsilon.dat','w')
            file_write.write(str(epsilon))
            file_write.close()
            torch.save(model.state_dict(),'results/model.ml')
        if frame_cnt%update_after_actions==0 and len(replay_mem['reward'])>batch_size:
            choices=np.random.choice( range( len(replay_mem['reward']) ), size=batch_size )
            STATE=[replay_mem['state'][id] for id in choices]
            NEXT_STATE=[replay_mem['next_state'][id] for id in choices]
            ACTION=[replay_mem['action'][id] for id in choices]
            REWARD=np.array([replay_mem['reward'][id] for id in choices])
            with torch.no_grad():
                a=M.predict(model_target,NEXT_STATE,device)
                TEMP,TEMP_INDX=torch.max(M.predict(model_target,NEXT_STATE,device),dim=1)
                REWARD_T=torch.tensor(REWARD)
                if torch.cuda.is_available():
                    REWARD_T=REWARD_T.to(torch.device("cuda:0")) 
                reward_true=torch.add(REWARD_T,gamma*TEMP)
            for epoch in range(epochs):
                M.train(model,reward_true,STATE,ACTION,device,optimizer)
        if frame_cnt%update_target_network==0:
            print(f'Updating N/W ...{frame_cnt}')
            model_target.load_state_dict(model.state_dict())
        if len(replay_mem['reward'])>mx_replay_memory:
            print('Deleting ...')
            del replay_mem['state'][:1]
            del replay_mem['next_state'][:1]
            del replay_mem['action'][:1]
            del replay_mem['reward'][:1]
        if terminated:
            break
    episode_reward_hist.append(episode_reward)
    episode_avg_reward.append(episode_reward/__)

# Save Relevant Results
np.save('results/ep_tot.npy',np.array(episode_reward_hist))
np.save('results/ep_avg.npy',np.array(episode_avg_reward))
torch.save(model.state_dict(),'results/model.ml')

### Test Model

In [ ]:
# Loading Results.
episode_reward_hist=np.load('results/ep_tot.npy')
episode_avg_reward=np.load('results/ep_avg.npy')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_tr=M.NNModel().to(device)
model_tr.load_state_dict(torch.load('results/model.ml'))

In [ ]:
def plotter(arr,xlabel,ylabel,title):
    plt.plot(arr)
    plt.xlabel(f'{xlabel}')
    plt.ylabel(f'{ylabel}')
    plt.title(f'{title}')
    plt.savefig(f'results/{title}.png')
    plt.show()
    print()

plotter(episode_reward_hist,'Episodes','Reward','Reward Per Episode VS Episodes')
plotter(episode_avg_reward,'Episodes','Avg Reward','Avg Reward Per Episode VS Episodes')

In [ ]:
def policy(observation,device):
    with torch.no_grad():
        action=torch.argmax(M.predict(model_tr,[observation],device))
        return action.item()

dbg=True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
env = SimpleGridWorld()
episodes=1
for _ in range(episodes):
    cumulative_reward,steps=0,0
    observation = env.reset()
    cnt=0
    while True:
        action = policy(copy.deepcopy(observation),device)  # User-defined policy function
        if dbg==True:
            env.printGrid()
            print(f'action {action}')
        observation, reward, terminated = env.action_step(action)
        cumulative_reward+=reward
        steps+=1
        if terminated:
            break
    print(f'Cumulative Reward ~ {cumulative_reward}; TimeTaken ~ {steps}')